In [358]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import NMF
from collections import defaultdict
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
#from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words

In [359]:
data = pd.read_csv('data/data_not_starbucks.csv').drop('Unnamed: 0', axis=1)

In [360]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 6 columns):
location.lat        474 non-null float64
location.lng        474 non-null float64
name                474 non-null object
combined_reviews    474 non-null object
num_review_words    474 non-null int64
final_address       474 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 22.3+ KB


### Filter out shops with less than 20 words in their review

In [361]:
over20_data = data[data['num_review_words'] >= 20].reset_index().drop('index', axis=1)

### Adding custom stopwords to sklearns defaults

In [362]:
stopwords = list(stop_words.ENGLISH_STOP_WORDS)

In [363]:
stopwords += ['coffee', 'shop', 'coffeeshop', 'starbucks', 'wa', 'seattle']

### Added LemmaTokenizer to lemmatize words 

In [364]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [365]:
tf = TfidfVectorizer(strip_accents='unicode',
                     tokenizer=LemmaTokenizer(),
                     stop_words=stopwords,
                     max_features=500)

In [366]:
tfidf = tf.fit_transform(over20_data['combined_reviews'])

In [367]:
words = tf.get_feature_names()

In [368]:
nmf = NMF(n_components=20)
nmf.fit(tfidf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [369]:
H = nmf.components_
W = nmf.transform(tfidf)

In [370]:
H.shape

(20, 500)

In [371]:
W.shape

(435, 20)

In [372]:
top_words_index = np.argsort(-H)[:,0:5]
most_common_words_per_topic = np.array(words)[top_words_index]
for i, items in enumerate(most_common_words_per_topic):
    print(i, items)

0 ['good' 'friendly' 'staff' 'cute' 'neighborhood']
1 ['sandwich' 'breakfast' 'delicious' 'soup' 'egg']
2 ['latte' 'best' 'espresso' 'mocha' 'white']
3 ['bagel' 'cheese' 'good' 'cream' 'breakfast']
4 ['crepe' 'like' 'wanted' 'stop' 'neighborhood']
5 ['cupcake' 'cake' 'ice' 'cream' 'pizza']
6 ['stand' 'time' 'barista' 'girl' 'drink']
7 ['lot' 'work' 'place' 'wifi' 'beer']
8 ['donut' 'doughnut' 'fashioned' 'pot' 'old']
9 ['croissant' 'pastry' 'good' 'bakery' 'baked']
10 ['chocolate' 'hot' 'mocha' 'dark' 'order']
11 ['great' 'service' 'love' 'place' 'people']
12 ['brew' 'bean' 'cold' 'espresso' 'pour']
13 ['owner' 'neighborhood' 'business' 'local' 'new']
14 ['market' 'pike' 'view' 'place' 'day']
15 ['waffle' 'beer' 'delicious' 'mean' 'try']
16 ['food' 'breakfast' 'egg' 'burrito' 'brunch']
17 ['biscuit' 'gravy' 'breakfast' 'worth' 'delicious']
18 ['card' 'free' 'drink' 'good' 'almond']
19 ['located' 'floor' 'building' 'store' 'center']


In [373]:
#def top_shops(W):
shop_dict = defaultdict(list)
for index, item in enumerate(W):
    key = np.argmax(item)
    value = item[key]
    name = over20_data['name'][index]
    shop_dict[key].append([value, name])
top_shop_names = defaultdict(list)
for feature in shop_dict:
    top_shop_names[feature] = list(np.sort(np.array(shop_dict[feature]).T)[1,-5:-1])
top_shop_names

defaultdict(list,
            {0: ['The Bridge Coffee House',
              'The Seattle Grind',
              'Tin Umbrella Coffee',
              'UW: Tower Café'],
             1: ['Seattle Aquarium Cafe',
              'The Station',
              'Treehouse Coffee',
              'Trinity Market'],
             2: ['Uptown Espresso',
              'Uptown Espresso Gameporium  - Delridge',
              'Vero Cafe',
              'Zeitgeist Kunst & Kaffee'],
             3: ['Fat Ducks Deli & Bakery',
              'Grateful Bread Baking Company & Cafe',
              "Lama G's",
              'Mabel Coffee'],
             4: ['Essential Bakery Cafe',
              'Joe Bar',
              'Le Petite Cafe',
              'Pearls Tea & Coffee'],
             5: ['Cupcake Royale and Verite Coffee',
              'Cupcake Royale and Vérité Coffee',
              'Pineapple Espresso & Bar',
              'The Yellow Leaf Cupcake Co'],
             6: ['UW: Husky Grind',
              '

In [357]:
%pdb

Automatic pdb calling has been turned OFF
